In [1]:
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.InteractiveSession(config=config)

In [2]:
# Importing data
dn_input = np.load('/home/beto/Documents/projects/DoggoNet/DN-dataset.npz')

x_train = dn_input['X_train']
x_test = dn_input['X_test']
y_train = dn_input['y_train']
y_test = dn_input['y_test']

print('Training set shape:', x_train.shape)
print('Testing set shape:', x_test.shape)

# Selecting an example
# etiqueta = dn_input['y_train'][1200]
# print(etiqueta[0])
# foto = dn_input['X_train'][1200]
# foto = np.multiply(foto, 255).astype(np.int8)
# Image.fromarray(foto, 'RGB').show()

# convert class
num_classes = 4
input_shape=(225, 300, 3)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Model definition
model = keras.Sequential([
    keras.Input(shape=input_shape),
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax"),
])

model.summary()

batch_size = 20
epochs = 12

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print('\n\n-------------------- MODEL COMPILED AND READY --------------------\n')

Training set shape: (3400, 225, 300, 3)
Testing set shape: (600, 225, 300, 3)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 223, 298, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 149, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 147, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 73, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 252288)            0         
_________________________________________________________________
dropout (Dropout)            (None, 252288)            0         
____________________________________________

In [3]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/12
 43/153 [=======>......................] - ETA: 2:10 - loss: 1.4557 - accuracy: 0.4640

KeyboardInterrupt: 